<a href="https://colab.research.google.com/github/adimyth/datascience_stuff/blob/master/nlp/BertTextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Data

In [1]:
import pandas as pd
from tensorflow.keras.utils import to_categorical

In [ ]:
!wget --no-check certificate "https://drive.google.com/uc?export=download&id=1fYV-PPmnJMkW5m9T1WoinDfI7x4P3xDV" -O Fake.zip

In [ ]:
!unzip Fake.zip

In [ ]:
!wget --no-check certificate "https://drive.google.com/uc?export=download&id=1VcIG3ZwM1Ab6v9_yYISMPvsWxufL4g2I" -O True.zip

In [ ]:
!unzip True.zip

* 1:True
* 0:False

In [6]:
true = pd.read_csv("True.csv")
true["target"] = [1]*true.shape[0]

In [7]:
fake = pd.read_csv("Fake.csv")
fake["target"] = [0]*fake.shape[0]

In [8]:
df = pd.concat([true, fake])

In [9]:
df = df.sample(frac=1).reset_index(drop=True)

In [10]:
df.head()

,title,text,subject,date,target
0,Campaign Chaos: Mike Pence Just Slammed Trump...,"Only a few days ago, it seemed as if Donald Tr...",News,"October 8, 2016",0
1,Trump defends 'star' tweet; Clinton says it's ...,NEW YORK (Reuters) - Republican presidential c...,politicsNews,"July 4, 2016",1
2,Putin Breaks Up With Trump In HUMILIATING Pub...,Ever since Donald Trump rode down the escalato...,News,"September 5, 2017",0
3,FLASHBACK: ‘The Syrian War: What You’re Not Be...,"21st Century Wire says Back in August 2013, Un...",Middle-east,"December 18, 2016",0
4,Trump says he had a 'very good talk' with Obam...,"PALM BEACH, Fla. (Reuters) - U.S. President-el...",politicsNews,"December 29, 2016",1


In [11]:
df['target'].value_counts()

0    23481
1    21417
Name: target, dtype: int64

# Fake News Classification

* [Fake and real news dataset | Kaggle](https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset)
* [Miachel Kazachok's Jigsaw Toxic Classification Kernel](https://www.kaggle.com/miklgr500/jigsaw-tpu-bert-with-huggingface-and-keras)


In [12]:
!pip install transformers

     |████████████████████████████████| 757kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 11.8MB/s 
     |████████████████████████████████| 3.0MB 30.0MB/s 
     |████████████████████████████████| 890kB 29.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ada291cbb24c2f53a2a4ff4af4d08d3700f27f528f8987b6ed470994c5b27c43
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [13]:
import os
import warnings

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers
from transformers import BertTokenizerFast
from tokenizers import BertWordPieceTokenizer
import traitlets

import matplotlib.pyplot as plt
from pprint import pprint 
import seaborn as sns

from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split

warnings.simplefilter("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [14]:
RANDOM_SEED = 42

In [15]:
sns.set_style("darkgrid")

Tokenizers library (implemented in rust) by Hugging Face provides a significantly faster BERT WordPiece tokenizer implementation than that in the Transformers library.

In [16]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    # truncates sentences longer than maxlen
    tokenizer.enable_truncation(max_length=maxlen)
    # right side padding till maxlen
    tokenizer.enable_padding(length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

BERT requires all 3 inputs `(ids, masks, and tokens)`. Huggingfact implementation allows us to pass only `ids` to the model & it still works. Also, `mask` and `tokentypeids` is useful for task with composite input

## Using Tokenizers Library

In [17]:
sentences = ["This is a test sentence",
             "Random sentences are difficult to write",
             "Maybe should have copied from Wikipedia",
             "Sentence one. Sentence two"
             ]

In [58]:
temp_encoding = fast_encode(pd.Series(sentences), fast_tokenizer)

The `fast_tokenizer` object is created below. So run that cell first

In [59]:
temp_encoding.shape

(4, 512)

In [60]:
temp_encoding[3][:30]

array([ 101, 6251, 2028, 1012, 6251, 2048,  102,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [61]:
print(" ".join(vocab['Word'].iloc[x] for x in temp_encoding[3][:30]), end="\n\n")    

[CLS] sentence one . sentence two [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]



## Using Transformer Library

* **Input IDs**: They are token indices, numerical representations of tokens building the sequences that will be used as input by the model.
* **Attention Mask**: is a binary tensor indicating the position of the padded indices so that the model does not attend to them.
* **Token Type IDs**: Some models require two different sequences to be encoded in same input ids. They are seperated using tokens like [SEP] & [CLS]
* **Segment IDs**: BERT requires understanding of where one sequence ends and where another begins. Segment IDs is a binary mask identifying the different sequences in the model.
* **Position IDs**: The position IDs are used by the model to identify which token is at which position.

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

In [20]:
print(tokenizer.encode_plus(sentences[0]))

{'input_ids': [101, 1188, 1110, 170, 2774, 5650, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [21]:
tokenizer.decode([101, 1188, 1110, 170, 2774, 5650, 102])

'[CLS] This is a test sentence [SEP]'

## Model

In [22]:
def build_model(transformer, loss='binary_crossentropy', max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    # last hidden state : (batch_size, sequence_length, hidden_size)
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    x = Dropout(0.35)(cls_token)
    out = Dense(2, activation='softmax')(x)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=3e-5), loss=loss, metrics=[tf.keras.metrics.AUC()])
    
    return model

For Classification task we are only interested in the BERT's output of `[CLS]` token.
BERT output is 3D of `(num_sentence, num_tokens, num_hidden_units)`. `[CLS]` is always the first token.

Since, we are interested in `[CLS]` tokens of all the sentences `[all_rows, [CLS], all_hidden_units]` so `[ :, 0, : ]`


![BERT](https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png)

## TPU Configs

In [ ]:
# Create strategy from tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [24]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [25]:
BATCH_SIZE

128

## Fast Tokenizer

In [ ]:
# First load the real tokenizer
tokenizer = transformers.BertTokenizerFast.from_pretrained('bert-base-uncased')

# Save the loaded tokenizer locally
save_path = 'distilbert_base_uncased/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
tokenizer.save_pretrained(save_path)

# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('distilbert_base_uncased/vocab.txt', lowercase=True)

**vocab.txt** - Stores index for each token

**special_tokens_map.json** - Stores special tokens & their meaning, such as -
* `[UNK]` - Unknown token
* `[SEP]` - Seperator token
* `[CLS]` - Classifier token used in BERT
* `[MASK]` - Token used for masking

**tokenizer_config.json** - Stores configuration option like lower casing, maximum length

## Train Test Split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], 
                                                    test_size=0.2, shuffle=True,
                                                    random_state=RANDOM_SEED)

In [28]:
X_train.shape, y_train.shape

((35918,), (35918,))

In [29]:
X_test.shape, y_test.shape

((8980,), (8980,))

## Encode

In [30]:
X_train_tokenized = fast_encode(X_train.astype(str), fast_tokenizer, maxlen=512)

In [31]:
X_train.shape, X_train_tokenized.shape

((35918,), (35918, 512))

In [32]:
out = open("distilbert_base_uncased/vocab.txt", "r").read().splitlines()
vocab = pd.DataFrame({"Word": out})

In [33]:
vocab.head()

,Word
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]


In [34]:
X_train_tokenized[10][:50]

array([  101,  2899,  1006, 26665,  1007,  1011,  1996,  2142,  2163,
        2006,  9317,  4161,  2167,  4420,  1037,  1523,  3078,  2769,
       28289,  5142,  1524,  1998,  2333,  2000,  2582,  3796,  2049,
        3754,  2000,  2224,  1996,  1057,  1012,  1055,  1012,  1998,
        2088,  3361,  3001,  2000,  4636,  2049,  4255,  3454,  1012,
        1996,  1057,  1012,  1055,  1012])

In [35]:
def print_texts(idx):
    print("="*30+"ORIGINAL TEXT"+"="*30)
    print(X_train.iloc[idx][:100], end="\n\n")
    print("="*30+"TOKENIZED FORM"+"="*30)
    print(X_train_tokenized[idx][:12], end="\n\n")
    print("="*30+"RECONSTRUCTED "+"="*30)
    print(" ".join(vocab['Word'].iloc[x] for x in X_train_tokenized[idx][:30]), end="\n\n")    

In [36]:
print_texts(0)

==============================ORIGINAL TEXT==============================
After only 16 hours, Microsoft was forced to pull the company s experimental  chat bot  from Twitter

==============================TOKENIZED FORM==============================
[ 101 2044 2069 2385 2847 1010 7513 2001 3140 2000 4139 1996]

==============================RECONSTRUCTED ==============================
[CLS] after only 16 hours , microsoft was forced to pull the company s experimental chat bot from twitter . microsoft s chat bot , which the company had nicknamed



In [37]:
X_test_tokenized = fast_encode(X_test.astype(str), fast_tokenizer, maxlen=512)

## Tensorflow Datasets

In [38]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_train_tokenized, to_categorical(y_train)))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

In [39]:
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(X_test_tokenized)
    .batch(BATCH_SIZE)
)

## Focal Loss

In [40]:
def focal_loss(gamma=2., alpha=.2):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

Positive and negative classes are fairly balanced here, so could use binary loss instead

In [41]:
%%time
with strategy.scope():
    transformer_layer = transformers.TFBertModel.from_pretrained('bert-base-uncased')
    model = build_model(transformer_layer, loss=focal_loss(gamma=1.5), max_len=512)
model.summary()

- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 512)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 512, 768), (None, 109482240 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
CPU times: user 21.5 s, sys: 12.4 s, total: 33.9 s
Wall time: 1min 13s


## LrScheduler

***Adam*** adapts to learning rate itself, however using additional learning rate decays can be helpful

In [42]:
def build_lrfn(lr_start=0.000001, lr_max=0.000002, 
               lr_min=0.0000001, lr_rampup_epochs=7, 
               lr_sustain_epochs=0, lr_exp_decay=.87):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

## Model Training

In [44]:
NUM_STEPS = X_train.shape[0] // BATCH_SIZE

In [45]:
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

train_history = model.fit(
    train_dataset,
    steps_per_epoch=NUM_STEPS,
    callbacks=[lr_schedule],
    epochs=3
)


Epoch 00001: LearningRateScheduler reducing learning rate to 1e-06.
Epoch 1/3


280/280 [==============================] - 155s 553ms/step - loss: 0.0505 - auc: 0.9576 - lr: 1.0000e-06

Epoch 00002: LearningRateScheduler reducing learning rate to 3.142857142857143e-06.
Epoch 2/3
280/280 [==============================] - 155s 554ms/step - loss: 0.0029 - auc: 0.9997 - lr: 3.1429e-06

Epoch 00003: LearningRateScheduler reducing learning rate to 5.285714285714285e-06.
Epoch 3/3
280/280 [==============================] - 155s 553ms/step - loss: 3.8754e-04 - auc: 1.0000 - lr: 5.2857e-06


In [46]:
y_pred = model.predict(test_dataset)

In [47]:
y_pred = np.argmax(y_pred, axis=1)

In [48]:
report_dict = classification_report(y_test, y_pred, output_dict=True)

In [49]:
pd.DataFrame(report_dict)

,0,1,accuracy,macro avg,weighted avg
precision,0.999376,0.998081,0.998775,0.998729,0.998776
recall,0.998339,0.999280,0.998775,0.998809,0.998775
f1-score,0.998857,0.998680,0.998775,0.998769,0.998775
support,4816.000000,4164.000000,0.998775,8980.000000,8980.000000


In [50]:
print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred)}")

ROC AUC Score: 0.9988092046683965


Test sentence for when Iran issued arrest warrant against Trump, taken from Hindustan Times page 

In [51]:
test_sentence = """Iran has issued an arrest warrant and asked Interpol for help in detaining President Donald Trump and dozens of others it believes carried out the drone strike that killed a top Iranian general in Baghdad, a local prosecutor reportedly said Monday.
While Trump faces no danger of arrest, the charges underscore the heightened tensions between Iran and the United States since Trump unilaterally withdrew America from Tehran’s nuclear deal with world powers.
Tehran prosecutor Ali Alqasimehr said Trump and more than 30 others whom Iran accuses of involvement in the Jan. 3 strike that killed Gen. Qassem Soleimani in Baghdad face “murder and terrorism charges,” the semiofficial ISNA news agency reported.
"""

In [52]:
test_encoded = fast_encode(pd.Series([test_sentence]).astype(str), fast_tokenizer)

In [53]:
test_encoded.shape

(1, 512)

In [54]:
model.predict(test_encoded)

array([[0.03456512, 0.9654349 ]], dtype=float32)

The model thinks it as a **TRUE** news.

US Election news taken from [NBC News](https://www.nbcnews.com/politics/2020-election/top-gop-senator-fears-trump-soft-independents-urges-shift-strategy-n1232024)

In [55]:
test_sentence = """
WASHINGTON — Senate Majority Whip John Thune sounded the alarm Wednesday that President Donald Trump's support among independent voters was "soft," urging a change in strategy in light of a New York Times survey that showed the president trailing Democrat Joe Biden by double digits.
The Times/Siena national poll found Biden winning 50 percent of registered voters, with Trump winning 36 percent. Among independents, Biden led by a substantial 18 points.
"Right now, obviously, Trump has a problem with the middle of the electorate, with independents, and they're the people who are undecided in national elections," Thune, R-S.D., told reporters in the Capitol. "I think he can win those back, but it'll probably require not only a message that deals with substance and policy but, I think, a message that conveys, perhaps, a different tone."
Trump won independent voters by 6 points in 2016, according to exit polls compiled by NBC News.
Asked whether the latest numbers were a wake-up call for the Trump campaign, Thune said, "It's a message that there needs to be a — certainly a change in probably strategy as far as the White House's messaging is concerned."
Thune said Trump could boost Republican Senate candidates if he could "perform better in terms of his own standing with the voters." He said the president is "in a bit of a low point right now, but as we all know in politics, in a short amount of time things can change."
"""

In [56]:
test_encoded = fast_encode(pd.Series([test_sentence]).astype(str), fast_tokenizer)

In [57]:
model.predict(test_encoded)

array([[0.09573054, 0.9042695 ]], dtype=float32)

The model thinks it as a true news with high percentage, which is correct!